In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

data_path = "../data-unversioned/p1ch7"

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4823, 0.4468),
                         (0.2470, 0.2435, 0.2616))
])

cifar10_train = datasets.CIFAR10(data_path, train=True, download=True, transform=transform)
cifar10_test  = datasets.CIFAR10(data_path, train=False, download=True, transform=transform)

trainX = torch.stack([img for img, _ in cifar10_train]).to(device)
trainY = torch.tensor([label for _, label in cifar10_train], dtype=torch.long).to(device)

testX = torch.stack([img for img, _ in cifar10_test]).to(device)
testY = torch.tensor([label for _, label in cifar10_test], dtype=torch.long).to(device)

train_dataset = torch.utils.data.TensorDataset(trainX, trainY)
val_dataset   = torch.utils.data.TensorDataset(testX, testY)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader   = torch.utils.data.DataLoader(val_dataset, batch_size=1000, shuffle=False)

Using device: cuda


In [2]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.act1  = nn.Tanh()
        self.pool1 = nn.MaxPool2d(2)

        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.act2  = nn.Tanh()
        self.pool2 = nn.MaxPool2d(2)

        self.fc1 = nn.Linear(32 * 8 * 8, 64)
        self.act3 = nn.Tanh()

        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        x = self.pool1(self.act1(self.conv1(x)))
        x = self.pool2(self.act2(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = self.act3(self.fc1(x))
        x = self.fc2(x)
        return x

model = Net().to(device)

In [3]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-3)

n_epochs = 300
start = time.time()

for epoch in range(n_epochs):
    model.train()
    running_loss = 0.0

    for batch_X, batch_Y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = loss_fn(outputs, batch_Y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * batch_X.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}, Loss: {epoch_loss:.4f}")

end = time.time()

print("\nTraining Time =", end - start, "seconds")
print("Final Training Loss =", epoch_loss)

model.eval()
correct = 0
total = 0

with torch.no_grad():
    for batch_X, batch_Y in val_loader:
        outputs = model(batch_X)
        preds = outputs.argmax(dim=1)
        correct += (preds == batch_Y).sum().item()
        total += batch_Y.size(0)

accuracy = correct / total
print("Evaluation Accuracy =", accuracy)

Epoch 1, Loss: 2.2468
Epoch 2, Loss: 2.1241
Epoch 3, Loss: 2.0291
Epoch 4, Loss: 1.9592
Epoch 5, Loss: 1.9086
Epoch 6, Loss: 1.8671
Epoch 7, Loss: 1.8284
Epoch 8, Loss: 1.7915
Epoch 9, Loss: 1.7574
Epoch 10, Loss: 1.7269
Epoch 11, Loss: 1.7000
Epoch 12, Loss: 1.6758
Epoch 13, Loss: 1.6537
Epoch 14, Loss: 1.6332
Epoch 15, Loss: 1.6139
Epoch 16, Loss: 1.5956
Epoch 17, Loss: 1.5782
Epoch 18, Loss: 1.5617
Epoch 19, Loss: 1.5459
Epoch 20, Loss: 1.5308
Epoch 21, Loss: 1.5166
Epoch 22, Loss: 1.5030
Epoch 23, Loss: 1.4902
Epoch 24, Loss: 1.4780
Epoch 25, Loss: 1.4665
Epoch 26, Loss: 1.4556
Epoch 27, Loss: 1.4452
Epoch 28, Loss: 1.4351
Epoch 29, Loss: 1.4253
Epoch 30, Loss: 1.4162
Epoch 31, Loss: 1.4071
Epoch 32, Loss: 1.3980
Epoch 33, Loss: 1.3893
Epoch 34, Loss: 1.3805
Epoch 35, Loss: 1.3719
Epoch 36, Loss: 1.3633
Epoch 37, Loss: 1.3548
Epoch 38, Loss: 1.3460
Epoch 39, Loss: 1.3375
Epoch 40, Loss: 1.3290
Epoch 41, Loss: 1.3206
Epoch 42, Loss: 1.3120
Epoch 43, Loss: 1.3034
Epoch 44, Loss: 1.29

In [6]:
# Homework 6-3A FNN Results
print("Homework 6-3A - FNN Results")
print("Training Time = 626.65065574646 seconds")
print("Final Training Loss = 0.49717244245529174")
print("Evaluation Accuracy = 0.4871")

Homework 6-3A - FNN Results
Training Time = 626.65065574646 seconds
Final Training Loss = 0.49717244245529174
Evaluation Accuracy = 0.4871
